# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [19]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [6]:
def simulate_single_flow(g, s, t,func,dono_insumo):
    functions = [random_geodesic_successor,random_path_successor,random_trail_successor, random_walk_successor ]
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            
            if(not dono_insumo):
                # Deixa de ser dono do insumo.
                g.nodes[n]['owner'] = False
                
                
            # Escolhe aleatoriamente um dos sucessores.
            try:
                m = functions[func-1](g, n)
            except IndexError:
                continue

            

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [7]:
def simulate_successful_flow(g, s, t,func,dono_insumo):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t,func,dono_insumo)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [8]:
def simulate_all_flows(g,func,dono_insumo):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t,func,dono_insumo)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Média de *closeness simulado* e *betweenness simulado* para muitas repetições da simulação acima.

In [9]:
def run(g,func,dono_insumo):
    TIMES = 100


    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g,func,dono_insumo)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES

    return pd.DataFrame({
        'família': [g.nodes[n]['label'] for n in g.nodes],
        'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
        'closeness analítico': [cc[n] for n in g.nodes],
        'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
        'betweenness analítico': [bc[n] for n in g.nodes],
    })

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [10]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

In [11]:
x=run(g,1, True)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.333333,0.333333,0.000000,0.000000
1,lambertes,0.325581,0.325581,0.000000,0.000000
2,albizzi,0.482759,0.482759,0.570549,0.212454
3,guadagni,0.466667,0.466667,0.690879,0.260073
4,pazzi,0.285714,0.285714,0.000000,0.000000
5,salviati,0.388889,0.388889,0.406593,0.142857
6,medici,0.560000,0.560000,1.388626,0.521978
7,tornabuon,0.482759,0.482759,0.247747,0.091575
8,bischeri,0.400000,0.400000,0.315220,0.120879
9,ridolfi,0.482759,0.482759,0.247088,0.086081


In [12]:
x=run(g,1, False)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.333333,0.333333,0.000000,0.000000
1,lambertes,0.325581,0.325581,0.000000,0.000000
2,albizzi,0.482759,0.482759,0.212473,0.212454
3,guadagni,0.466667,0.466667,0.259286,0.260073
4,pazzi,0.285714,0.285714,0.000000,0.000000
5,salviati,0.388889,0.388889,0.142857,0.142857
6,medici,0.560000,0.560000,0.520549,0.521978
7,tornabuon,0.482759,0.482759,0.089286,0.091575
8,bischeri,0.400000,0.400000,0.121374,0.120879
9,ridolfi,0.482759,0.482759,0.087527,0.086081


In [13]:
x=run(g,2, True)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.228813,0.333333,0.389615,0.000000
1,lambertes,0.232631,0.325581,0.400714,0.000000
2,albizzi,0.256366,0.482759,0.615330,0.212454
3,guadagni,0.265231,0.466667,0.744121,0.260073
4,pazzi,0.203574,0.285714,0.260055,0.000000
5,salviati,0.226088,0.388889,0.438901,0.142857
6,medici,0.266654,0.560000,0.824011,0.521978
7,tornabuon,0.277927,0.482759,0.685110,0.091575
8,bischeri,0.263148,0.400000,0.648571,0.120879
9,ridolfi,0.276277,0.482759,0.697912,0.086081


In [14]:
x=run(g,2, False)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.200768,0.333333,0.000000,0.000000
1,lambertes,0.213039,0.325581,0.000000,0.000000
2,albizzi,0.258506,0.482759,0.229176,0.212454
3,guadagni,0.272841,0.466667,0.405604,0.260073
4,pazzi,0.200187,0.285714,0.000000,0.000000
5,salviati,0.246578,0.388889,0.142857,0.142857
6,medici,0.308291,0.560000,0.629066,0.521978
7,tornabuon,0.253055,0.482759,0.334231,0.091575
8,bischeri,0.245404,0.400000,0.371758,0.120879
9,ridolfi,0.255282,0.482759,0.365440,0.086081


In [15]:
x=run(g,3, True)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.222360,0.333333,0.372253,0.000000
1,lambertes,0.225391,0.325581,0.395769,0.000000
2,albizzi,0.250886,0.482759,0.683791,0.212454
3,guadagni,0.252618,0.466667,0.906099,0.260073
4,pazzi,0.199141,0.285714,0.267418,0.000000
5,salviati,0.220291,0.388889,0.426923,0.142857
6,medici,0.254153,0.560000,1.266374,0.521978
7,tornabuon,0.268473,0.482759,0.935769,0.091575
8,bischeri,0.249507,0.400000,0.779176,0.120879
9,ridolfi,0.261375,0.482759,0.928297,0.086081


In [16]:
x=run(g,3, False)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.217556,0.333333,0.000000,0.000000
1,lambertes,0.205317,0.325581,0.000000,0.000000
2,albizzi,0.277201,0.482759,0.265934,0.212454
3,guadagni,0.271042,0.466667,0.406978,0.260073
4,pazzi,0.195894,0.285714,0.000000,0.000000
5,salviati,0.245076,0.388889,0.142857,0.142857
6,medici,0.288372,0.560000,0.698132,0.521978
7,tornabuon,0.251552,0.482759,0.319560,0.091575
8,bischeri,0.227487,0.400000,0.336264,0.120879
9,ridolfi,0.249165,0.482759,0.332033,0.086081


In [17]:
x=run(g,4, True)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.140061,0.333333,0.799560,0.000000
1,lambertes,0.142208,0.325581,0.706429,0.000000
2,albizzi,0.163050,0.482759,3.096538,0.212454
3,guadagni,0.164216,0.466667,4.602088,0.260073
4,pazzi,0.127884,0.285714,0.757582,0.000000
5,salviati,0.143189,0.388889,1.795714,0.142857
6,medici,0.166193,0.560000,6.976703,0.521978
7,tornabuon,0.166991,0.482759,2.426154,0.091575
8,bischeri,0.151557,0.400000,2.770000,0.120879
9,ridolfi,0.166799,0.482759,2.563077,0.086081


In [18]:
x=run(g,4, False)
x

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
0,ginori,0.037153,0.333333,0.745165,0.000000
1,lambertes,0.037099,0.325581,0.748132,0.000000
2,albizzi,0.036254,0.482759,2.431319,0.212454
3,guadagni,0.034398,0.466667,3.261758,0.260073
4,pazzi,0.040559,0.285714,0.728956,0.000000
5,salviati,0.038487,0.388889,1.569945,0.142857
6,medici,0.036578,0.560000,5.009890,0.521978
7,tornabuon,0.036610,0.482759,2.447527,0.091575
8,bischeri,0.035384,0.400000,2.414286,0.120879
9,ridolfi,0.034835,0.482759,2.446593,0.086081


Construção de data frame só para comparar mais facilmente.

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.